In [ ]:
model_dir = 'models' # temporary working directory for storing the prediction models
sim_id = 'zamg'
sim_dir = 'simulations' # directory for storing the simulation results

# start and configure tensorflow
# check whether GPU-enabled tensorflow is active; if not, try setting LD_LIBRARY_PATH
# %env LD_LIBRARY_PATH=/home/alex/miniconda3/envs/tf/lib/
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
for device in physical_devices:
    # needed for KerasTuner
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
from typing import Dict, List
from base.training import zamg_dense, zamg_lstm, zamg_conv_lstm, ModelDef
from base.window_generator import WindowGenerator

# Specify the model architectures for the simulations
model_defs: List[ModelDef] = [
    ModelDef(name='simple_dense',
             model_fn=zamg_dense,
             optimizer='rmsprop',
             learning_rate=0.002399431372613329,
             fine_tune_rate=2e-5,
             freeze_layers=['dense1'],
             ),
    ModelDef(name='simple_lstm',
             model_fn=zamg_lstm,
             optimizer='rmsprop',
             learning_rate=0.0005028709561526049,
             fine_tune_rate=5e-6,
             freeze_layers=['lstm'],
             ),
    ModelDef(name='conv_lstm',
             model_fn=zamg_conv_lstm,
             optimizer='adam',
             learning_rate=0.00024407154977751656,
             fine_tune_rate=2e-6,
             freeze_layers=['conv1', 'conv2', 'conv3', 'conv4'],
             )
    ]

In [ ]:
from typing import Tuple
from base.simulator_data import SimulatorData


# Define all the training datasets
def vienna_2010_2019() -> (SimulatorData, WindowGenerator):
    sim_data = SimulatorData(
        initial_source= 'zamg_vienna_hourly.pickle',
        initial_start='2010-01-01',
        initial_end='2019-12-31',
        continual_start='2020-01-01',
        continual_end='2021-12-31',
        )

    window = sim_data.get_window_generator(
        input_features=['TL', 'P', 'RF', 'SO'],
        output_features=['TL'],
        periodicity=['day', 'year'],
        input_length=24 * 5,
        output_length=24,
        stride=6,
        sampling_rate=6,
        validation_split='730d',
        )
    return sim_data, window


def vienna_2019_2019():
    sim_data = SimulatorData(
        initial_source='zamg_vienna_hourly.pickle',
        initial_start='2019-01-01',
        initial_end='2019-12-31',
        continual_start='2020-01-01',
        continual_end='2021-12-31',
        )

    window = sim_data.get_window_generator(
        input_features=['TL', 'P', 'RF', 'SO'],
        output_features=['TL'],
        periodicity=['day', 'year'],
        input_length=24 * 5,
        output_length=24,
        stride=6,
        sampling_rate=6,
        validation_split='110d',  # 365 * 0.3
        )
    return sim_data, window


def vienna_201907_201912():
    sim_data = SimulatorData(
        initial_source='zamg_vienna_hourly.pickle',
        initial_start='2019-07-01',
        initial_end='2019-12-31',
        continual_start='2020-01-01',
        continual_end='2021-12-31',
        )

    window = sim_data.get_window_generator(
        input_features=['TL', 'P', 'RF', 'SO'],
        output_features=['TL'],
        periodicity=['day', 'year'],
        input_length=24 * 5,
        output_length=24,
        stride=6,
        sampling_rate=6,
        validation_split='55d',  # 183 * 0.3
        )
    return sim_data, window


def linz_2010_2019():
    sim_data = SimulatorData(
        initial_source='zamg_linz_hourly.pickle',
        initial_start='2010-01-01',
        initial_end='2019-12-31',
        continual_source='zamg_vienna_hourly.pickle',
        continual_start='2020-01-01',
        continual_end='2021-12-31',
        )

    window = sim_data.get_window_generator(
        input_features=['TL', 'P', 'RF', 'SO'],
        output_features=['TL'],
        periodicity=['day', 'year'],
        input_length=24 * 5,
        output_length=24,
        stride=6,
        sampling_rate=6,
        validation_split='730d',
        )
    return sim_data, window


training_data: Dict[str, Tuple[SimulatorData, WindowGenerator]] = {
    'vienna_2010_2019': vienna_2010_2019(),
    'vienna_2019_2019': vienna_2019_2019(),
    'vienna_201907_201912': vienna_201907_201912(),
    'linz_2010_2019': linz_2010_2019(),
    }

In [ ]:
import os.path
from base.training import train_model


# First we train all required models
def get_model_id(data_id: str, model_name: str):
    return f'{sim_id}_{data_id}_{model_name}'


os.makedirs(model_dir, exist_ok=True)
for model_def in model_defs:
    for data_id, (sim_data, window) in training_data.items():
        model_id = get_model_id(data_id, model_def.name)

        if os.path.exists(os.path.join(model_dir, model_id)):
            print(f'Skipping already existing model: {model_id}')
        else:
            print(f'Next model: {model_id}')
            train_model(window,
                        model_def.model_fn,
                        max_epochs=100,
                        patience=20,
                        model_dir=model_dir,
                        model_name=model_id,
                        )

In [ ]:
from base.meta_simulator import MetaSimulator

# Then we run the simulations for every trained model
os.makedirs(sim_dir, exist_ok=True)
for model_def in model_defs:
    strategies = {
        'static': {
            'cl_strategy': {'type': 'NoUpdateStrategy', 'object': {}},
            'deploy_strategy': {'type': 'DeployOnceStrategy', 'object': {}}},
        'retrain_short': {
            'cl_strategy': {'type': 'RetrainStrategy',
                            'object': {'epochs': 100, 'patience': 20,
                                       'optimizer': model_def.optimizer, 'learning_rate': model_def.learning_rate,
                                       'stride': 1, 'validation': 0.3}},
            'deploy_strategy': {'type': 'FixedIntervalStrategy', 'object': {'interval': 60 * 60 * 24 * (366 / 4)}}},
        'retrain_long': {
            'cl_strategy': {'type': 'RetrainStrategy',
                            'object': {'epochs': 100, 'patience': 20,
                                       'optimizer': model_def.optimizer, 'learning_rate': model_def.learning_rate,
                                       'stride': 1, 'validation': 0.3}},
            'deploy_strategy': {'type': 'FixedIntervalStrategy', 'object': {'interval': 60 * 60 * 24 * (366 / 2)}}},
        'fine_tune_short': {
            'cl_strategy': {'type': 'TransferLearningStrategy',
                            'object': {'freeze_layers': [], 'epochs': 100,
                                       'optimizer': model_def.optimizer, 'learning_rate': model_def.fine_tune_rate,
                                       'stride': 1, 'validation': '4w'}},
            'deploy_strategy': {'type': 'FixedIntervalStrategy', 'object': {'interval': 60 * 60 * 24 * (366 / 4)}}},
        'fine_tune_long': {
            'cl_strategy': {'type': 'TransferLearningStrategy',
                            'object': {'freeze_layers': [], 'epochs': 100,
                                       'optimizer': model_def.optimizer, 'learning_rate': model_def.fine_tune_rate,
                                       'stride': 1, 'validation': '8w'}},
            'deploy_strategy': {'type': 'FixedIntervalStrategy', 'object': {'interval': 60 * 60 * 24 * (366 / 2)}}},
        'transfer_short': {
            'cl_strategy': {'type': 'TransferLearningStrategy',
                            'object': {'freeze_layers': model_def.freeze_layers, 'epochs': 100,
                                       'optimizer': model_def.optimizer, 'learning_rate': model_def.learning_rate,
                                       'stride': 1, 'validation': '4w'}},
            'deploy_strategy': {'type': 'FixedIntervalStrategy', 'object': {'interval': 60 * 60 * 24 * (366 / 4)}}},
        'transfer_long': {
            'cl_strategy': {'type': 'TransferLearningStrategy',
                            'object': {'freeze_layers': model_def.freeze_layers, 'epochs': 100,
                                       'optimizer': model_def.optimizer, 'learning_rate': model_def.learning_rate,
                                       'stride': 1, 'validation': '8w'}},
            'deploy_strategy': {'type': 'FixedIntervalStrategy', 'object': {'interval': 60 * 60 * 24 * (366 / 2)}}}}

    for data_id, (sim_data, window) in training_data.items():
        for strategy in ['retrain_short', 'retrain_long']:
            use_initial_data = data_id != 'linz_2010_2019'
            strategies[strategy]['cl_strategy']['object']['use_initial_data'] = use_initial_data

        model_id = get_model_id(data_id, model_def.name)
        meta_sim = MetaSimulator(
            simulator_data=sim_data.to_dict(),
            strides=[
                600,  # 10 minutes
                1800,  # 30 minutes
                3600,  # 1 hour
                ], threshold_metrics={
                'TL_high': {'type': 'L2Threshold',
                            'object': {'threshold': 2.5, 'measurement_indices': [0], 'prediction_indices': [0]}},
                'TL_medium': {'type': 'L2Threshold',
                              'object': {'threshold': 1.0, 'measurement_indices': [0], 'prediction_indices': [0]}},
                'TL_low': {'type': 'L2Threshold',
                           'object': {'threshold': 0.5, 'measurement_indices': [0], 'prediction_indices': [0]}}
                },
            strategies=strategies,
            model_dir=model_dir,
            base_model_id=model_id,
            result_dir=os.path.join(sim_dir, model_id)
            )
        meta_sim.run_simulations(verbose=1,
                                 store_data=False,
                                 callback=tf.keras.backend.clear_session)
        meta_sim.run_repeat_simulations(verbose=2)